<a href="https://colab.research.google.com/github/nxxk23/BasicProg_DataViz65/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/DPDM/Project/bank-full.csv')
data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [5]:
data= data[['age', 'job', 'marital', 'education', 'default', 'balance', 'housing','loan', 'contact', 'day', 'month', 'duration', 'campaign','poutcome', 'y']]

In [6]:
data.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'poutcome',
       'y'],
      dtype='object')

#replace value in columns


In [7]:
data['poutcome'].unique()

array(['unknown', 'failure', 'other', 'success'], dtype=object)

In [8]:
data['job'].replace({'management':'0', 'technician':'1', 'entrepreneur':'2', 'blue-collar':'3',
       'unknown':'4', 'retired':'5', 'admin.':'6', 'services':'7', 'self-employed':'8',
       'unemployed':'9', 'housemaid':'10', 'student':'11'}, inplace=True)

data['marital'].replace({'married':'2', 'single':'1', 'divorced':'0'}, inplace=True)

data['education'].replace({'tertiary':'3', 'secondary':'2', 'unknown':'0', 'primary':'1'}, inplace=True)

data['default'].replace({'no':'0', 'yes':'1'}, inplace=True)

data['housing'].replace({'no':'0', 'yes':'1'}, inplace=True)

data['loan'].replace({'no':'0', 'yes':'1'}, inplace=True)

data['contact'].replace({'unknown':'0', 'cellular':'1', 'telephone':'2'}, inplace=True)

data['month'].replace({'may':'5', 'jun':'6', 'jul':'7', 'aug':'8', 'oct':'10', 'nov':'11', 'dec':'12', 'jan':'1', 'feb':'2',
       'mar':'3', 'apr':'4', 'sep':'9'}, inplace=True)

data['poutcome'].replace({'unknown':'0', 'failure':'1', 'other':'3', 'success':'2'}, inplace=True)

<ipython-input-8-0c1b9775cfc5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['job'].replace({'management':'0', 'technician':'1', 'entrepreneur':'2', 'blue-collar':'3',
<ipython-input-8-0c1b9775cfc5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['marital'].replace({'married':'2', 'single':'1', 'divorced':'0'}, inplace=True)
<ipython-input-8-0c1b9775cfc5>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['education'].

In [9]:
data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,poutcome,y
0,58,0,2,3,0,2143,1,0,0,5,5,261,1,0,no
1,44,1,1,2,0,29,1,0,0,5,5,151,1,0,no
2,33,2,2,2,0,2,1,1,0,5,5,76,1,0,no
3,47,3,2,0,0,1506,1,0,0,5,5,92,1,0,no
4,33,4,1,0,0,1,0,0,0,5,5,198,1,0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,1,2,3,0,825,0,0,1,17,11,977,3,0,yes
45207,71,5,0,1,0,1729,0,0,1,17,11,456,2,0,yes
45208,72,5,2,2,0,5715,0,0,1,17,11,1127,5,2,yes
45209,57,3,2,2,0,668,0,0,2,17,11,508,4,0,no


In [10]:
# separate the features and target variable
X = data.drop('y', axis=1)
y = data['y']

## Decision tree

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale the features if necessary
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# build a decision tree classifier and fit it to the training data
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train_scaled, y_train)

# make predictions on the testing data and calculate accuracy
y_pred = clf.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8696229127501935
